# Import SSBClient
Import the SSBClient class for querying the SSB API.

In [28]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../common/ssb')
from client import SSBClient

# Query SSB API with patients.json
Load the query from patients.json and fetch the data using SSBClient.

In [18]:
import json
with open('../../config/queries/patients.json', 'r') as f:
    patients_query = json.load(f)

client = SSBClient()
df_patients = client.query('10261', patients_query)
df_patients.head()

,Region,Diagnose3,ContentsCode,Tid,value
0,Total,Acute respiratory infections,Number of in-patient stays (discharges),2012,11083
1,Total,Acute respiratory infections,Number of in-patient stays (discharges),2013,10589
2,Total,Acute respiratory infections,Number of in-patient stays (discharges),2014,9937
3,Total,Acute respiratory infections,Number of in-patient stays (discharges),2015,9033
4,Total,Acute respiratory infections,Number of in-patient stays (discharges),2016,9783


In [19]:
df_patients.drop(columns=['Diagnose3'], inplace=True)
df_patients.rename(columns={'Region': 'county', 'ContentsCode': 'variable', 'value': 'patient_count', 'Tid': "year"}, inplace=True)

In [22]:
df_patients.to_csv('../../data/patients.csv', index=False)

In [32]:
df_patients = pd.read_csv('../../data/patients.csv')

In [33]:
# Create unique region IDs and reduced IDs for merged regions
import numpy as np

region_map = {
    'Total': 'total',
    'Østfold': 'ostfold',
    'Akershus': 'akershus',
    'Viken (2020-2023)': 'viken',
    'Østfold (-2019)': 'ostfold_old',
    'Akershus (-2019)': 'akershus_old',
    'Oslo': 'oslo',
    'Innlandet': 'innlandet',
    'Hedmark (-2019)': 'hedmark_old',
    'Oppland (-2019)': 'oppland_old',
    'Buskerud': 'buskerud',
    'Buskerud (-2019)': 'buskerud_old',
    'Vestfold': 'vestfold',
    'Telemark': 'telemark',
    'Vestfold og Telemark (2020-2023)': 'vestfold_telemark',
    'Vestfold (-2019)': 'vestfold_old',
    'Telemark (-2019)': 'telemark_old',
    'Agder': 'agder',
    'Aust-Agder (-2019)': 'aust_agder_old',
    'Vest-Agder (-2019)': 'vest_agder_old',
    'Rogaland': 'rogaland',
    'Vestland': 'vestland',
    'Hordaland (-2019)': 'hordaland_old',
    'Sogn og Fjordane (-2019)': 'sogn_fjordane_old',
    'Møre og Romsdal': 'more_romsdal',
    'Trøndelag - Trööndelage': 'trondelag',
    'Sør-Trøndelag (-2017)': 'sor_trondelag_old',
    'Nord-Trøndelag (-2017)': 'nord_trondelag_old',
    'Nordland - Nordlánnda': 'nordland',
    'Troms - Romsa - Tromssa': 'troms',
    'Finnmark - Finnmárku - Finmarkku': 'finnmark',
    'Troms og Finnmark - Romsa ja Finnmárku (2020-2023)': 'troms_finnmark',
    'Troms - Romsa (-2019)': 'troms_old',
    'Finnmark - Finnmárku (-2019)': 'finnmark_old',
    'Unknown': 'unknown'
}

# Assign unique region IDs
df_patients['county_id'] = df_patients['county'].map(region_map)

# Reduced region ID: combine merged regions
def reduce_region(region):
    if region in ['Østfold', 'Østfold (-2019)']:
        return 'ostfold'
    if region in ['Akershus', 'Akershus (-2019)']:
        return 'akershus'
    if region in ['Buskerud', 'Buskerud (-2019)']:
        return 'buskerud'
    if region in ['Vestfold', 'Vestfold (-2019)']:
        return 'vestfold'
    if region in ['Telemark', 'Telemark (-2019)']:
        return 'telemark'
    if region in ['Troms - Romsa - Tromssa', 'Troms - Romsa (-2019)']:
        return 'troms'
    if region in ['Finnmark - Finnmárku - Finmarkku', 'Finnmark - Finnmárku (-2019)']:
        return 'finnmark'
    if region in ['Sør-Trøndelag (-2017)', 'Nord-Trøndelag (-2017)', 'Trøndelag - Trööndelage']:
        return 'trondelag'
    return region_map.get(region, 'unknown')

df_patients['county_reduced'] = df_patients['county'].apply(reduce_region)

In [39]:
df_patients["variable"] = df_patients.variable.map({'Number of in-patient stays (discharges)': 'inpatient_stays', 'Number of out-patient consultations': 'outpatient_consultations'})

In [40]:
# pivot longer at variable
df_patients = df_patients.pivot_table(index=['year', 'county', 'county_id', 'county_reduced'], columns='variable', values='patient_count', fill_value=0).reset_index()
# aggregate to yearly counts by reduced county
df_patients = df_patients.groupby(['year', 'county_reduced']).agg({'inpatient_stays': np.sum, 'outpatient_consultations': np.sum}).reset_index()

/var/folders/56/097r5m_53z97wd0wwmqws76m0000gn/T/ipykernel_66207/1898915539.py:4: FutureWarning: The provided callable <function sum at 0x114844680> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_patients = df_patients.groupby(['year', 'county_reduced']).agg({'inpatient_stays': np.sum, 'outpatient_consultations': np.sum}).reset_index()
/var/folders/56/097r5m_53z97wd0wwmqws76m0000gn/T/ipykernel_66207/1898915539.py:4: FutureWarning: The provided callable <function sum at 0x114844680> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_patients = df_patients.groupby(['year', 'county_reduced']).agg({'inpatient_stays': np.sum, 'outpatient_consultations': np.sum}).reset_index()


In [42]:
df_patients.county_reduced.value_counts()

county_reduced
agder                13
akershus             13
aust_agder_old       13
buskerud             13
finnmark             13
hedmark_old          13
hordaland_old        13
innlandet            13
more_romsdal         13
nordland             13
oppland_old          13
oslo                 13
ostfold              13
rogaland             13
sogn_fjordane_old    13
telemark             13
total                13
troms                13
troms_finnmark       13
trondelag            13
unknown              13
vest_agder_old       13
vestfold             13
vestfold_telemark    13
vestland             13
viken                13
Name: count, dtype: int64